## Imports

In [1]:
import nltk.data
from fuzzywuzzy import fuzz

/Users/tssuser/anaconda/lib/python3.5/site-packages/fuzzywuzzy/fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Questions

In [3]:
question_dict = {
0 : 'How strongly do you agree or disagree with our proposed environmental objective for the next regime?',
1 : 'Any additional comments on our proposed environmental objective for the next regime?',
2 : 'How strongly do you agree or disagree with our proposal for the length of the next regime?',
3 : 'Any additional comments on our proposal for the length of the regime?',
4 : 'How strongly do you agree or disagree with our proposal to introduce a new QC/0.125 category for aircraft between 81 and 83.9 EPNdB?',
5 : 'How strongly do you agree or disagree with our proposal for all aircraft quieter than this to remain QC/0 but count towards the airports movement limit?',
6 : 'Any additional comments on our proposals for the Quota Count System?',
7 : 'How strongly do you agree or disagree with the proposal for movement limits to remain unchanged at Heathrow?',
8 : 'Any additional comments on our proposal for Heathrow movement limits?',
9 : 'How strongly do you agree or disagree with the proposal for movement limits to remain unchanged at Gatwick?',
10 : 'Any additional comments on our proposal for Gatwick’s movement limit?',
11 : 'How strongly do you agree or disagree with the proposal to raise Stansted’s movement limits to reflect the current number of exempt aircraft in operation?',
12 : 'Any additional comments on our proposal for Stansted’s movement limit?',
13 : 'How strongly do you agree or disagree with our proposals to encourage the use of quieter aircraft at Heathrow?',
14 : 'Any additional comments on how you feel noise quotas can be set in order to encourage the use of quieter aircraft at Heathrow?',
15 : 'How strongly do you agree or disagree with our proposals to encourage the use of quieter aircraft at Gatwick?',
16 : 'Any additional comments on how you feel noise quotas can best be set in order to encourage the use of quieter aircraft at Gatwick?',
17 : 'How strongly do you agree or disagree with our proposals to encourage the use of quieter aircraft at Stansted?',
18 : 'Any additional comments on how you feel noise quotas can best be set in order to encourage the use of quieter aircraft at Stansted?',
19 : 'Q10 Any further views on our proposals, or their potential impact on governments ability to fulfil requirements of the Public Sector Equality Duty.',
20 : 'What additional evidence do you have on the expected costs and benefits of the policy options outlined in sections 5 and 6 of the IA.',
21 : 'What additional evidence do you have on the assumptions to develop the unconstrained scenario and the policy scenarios outlined in section 7.1.2. In particular, any additional evidence on how airports would make use of the carryover and overrun flexibility and which flights would be affected when and airport hits either its movement or quota limit. Please email any attachments to night.flights@dft.gsi.gov.uk',
22 : 'What additional evidence do you have on how airlines that have ordered new aircraft types (e.g. Airbus A320neo and Boeing 737 Max) plan on introducing these into their fleets, as outlined in section 7.1.4.',
23 : 'What additional evidence do you have on airline responses to a reduction in night flights regarding flight cancellations and rescheduling, and how passengers might respond to these changes, as outlined in sections 6.2.6 and 7.3.',
24 : 'What additional evidence do you have on the likely amount of time needed for stakeholders to read and understand the regulations, as outlined in section 8.2.2.',
25 : 'What additional evidence do you have on the monetised impact on business of the policy options outlined in section 10.',
26 : 'What additional evidence do you have on the economic and wider impacts of changes to the night flights regime outlined in section 11.',
27 : 'What additional evidence do you have on the impact of our policy options on competition, as outlined in section 11.1.',
28 : 'What additional evidence do you have of the policy impacts on small and micro businesses, outlined in section 11.2.' ,
29 : 'What additional evidence do you have on which policy option is best suited to achieve the environmental objective as outlined in section 12.'
}
question_list = list(question_dict.values())

## NLP

In [4]:
# Splits the responses into sentences

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
fp = open('/Users/tssuser/work/Night-Flight/data/sample.txt')
data = fp.read()
clean = '\n-----\n'.join(tokenizer.tokenize(data))
split_list = clean.split('\n-----\n')

## Question Text Finder

In [8]:
# Finds the sentences that most likely relate to questions and returns a list containing: [Index of the question, % Match, Start Index for the answer, End Index for the answer]

matches1 = {}
index_list = []

for key in question_dict.keys():
    
    matches2 = {}
    i = 0
    
    for sentence in split_list:
        x = fuzz.ratio(question_dict[key],sentence) 
        matches2[i] = x
        i += 1
        
    calc = max(matches2, key=matches2.get)
    maximum = [calc,matches2[calc]]
    start_index = max(matches2, key=matches2.get) +1
    maximum.append(start_index)
    matches1[key] = maximum

matches3 = {}
    
for key, value in matches1.items():
    if value[1] >= 80:
        matches3[key] = matches1[key]
        index_list.append(matches1[key][0])
    else:
        pass

index_list.pop(0)

i2 = 0

for key, value in matches3.items():
    try:
        value.append(index_list[i2] - 1)        
        i2 +=1
    except IndexError:
        value.append(len(split_list) - 1)
    

In [9]:
matches3

{0: [25, 100, 26, 34],
 1: [35, 92, 36, 49],
 2: [50, 100, 51, 54],
 3: [55, 91, 56, 60],
 4: [61, 100, 62, 64],
 5: [65, 100, 66, 70],
 6: [71, 88, 72, 73],
 7: [74, 100, 75, 77],
 8: [78, 89, 79, 81],
 9: [82, 100, 83, 98],
 10: [99, 91, 100, 107],
 11: [108, 100, 109, 110],
 12: [111, 91, 112, 114],
 13: [115, 100, 116, 118],
 14: [119, 93, 120, 122],
 15: [123, 100, 124, 145],
 16: [146, 95, 147, 161],
 17: [162, 100, 163, 165],
 18: [166, 95, 167, 169],
 19: [170, 91, 171, 183]}

## Splitting the Response into Questions

In [10]:
responses = {}

for key, value in matches3.items():
    
    i = value[2]
    data = []
    
    while i <= value[3]:
        data.append(split_list[i])
        i+=1
    
    responses[key] = data

In [11]:
responses

{0: ['(Encourage the use of quieter aircraft to limit or reduce the number of people significantly affected by aircraft noise at night, while maintaining the existing benefits of night flights.)',
  '6.',
  'We welcome the aim to encourage quieter aircraft.',
  'However the aim should be to reduce the number of people significantly affected – delete the weasel words ‘to limit’ (which in DfT speak can mean to increase but not too much).',
  '7.',
  'It should be made clear that this objective can only be achieved by quieter aircraft not by merely introducing more concentrated flight paths.',
  '8.',
  'The words ‘existing benefits’ are discussed in paragraphs 9-13 below.',
  'Q1b.'],
 1: ['9.',
  'Although the consultation refers to the existing benefits of night flights there is no attempt to quantify them.',
  'It is stated in paragraph 1 that aviation ‘directly supports around 230,000 jobs … and contributes over £21 bn annually to UK GDP.’ but that is exaggerated and misleading:  it 